# 🎓 이커머스 이탈 예측 프로젝트 (Final Mentor Ver.)
**"데이터 무결성", "초단기 데이터 대응", "학습 안정성"이 모두 적용된 최종본**

### 🛠️ 수정 이력 (Fix History)
1.  **Memory Load Logic**: 구글 드라이브 마운트 및 로컬 파일 탐색 로직 강화.
2.  **Adaptive Time-Split**: 데이터 기간이 짧아도(12일 등) 자동으로 학습/검증 구간을 설정하는 알고리즘 적용.
3.  **Empty Data Safety**: 데이터 부족으로 0건이 되어도 에러 없이 더미 데이터를 생성해 파이프라인 유지.


## [Step 1] 환경 설정 및 한글 폰트

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
import os, warnings

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler
from torch.optim.lr_scheduler import StepLR
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, f1_score, precision_recall_curve

warnings.filterwarnings('ignore')

def set_korean_font():
    font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
    if not os.path.exists(font_path):
        os.system("sudo apt-get -qq -y install fonts-nanum")
        os.system("sudo fc-cache -fv")
        os.system("rm ~/.cache/matplotlib -rf")
    try:
        fm.fontManager.addfont(font_path)
        plt.rc('font', family=fm.FontProperties(fname=font_path).get_name())
    except:
        plt.rc('font', family='NanumBarunGothic')
    plt.rcParams['axes.unicode_minus'] = False

set_korean_font()
print("✅ 폰트 설정 완료")


## [Step 2] 데이터 로드 (Robust)

In [ ]:

print("▶ 데이터 로딩 중...")

# 1. Drive Mount
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    possible_paths = [
        '/content/drive/MyDrive/Deep-Learning/프로젝트',
        '/content/drive/MyDrive/Colab Notebooks',
        '/content/drive/MyDrive'
    ]
    for p in possible_paths:
        if os.path.exists(p):
            os.chdir(p)
            print(f"📂 경로 이동: {p}")
            break
except:
    print("⚠️ 로컬/드라이브 실패 (현재 경로 사용)")

# 2. File Check
target_file = 'clock_all.csv'
if not os.path.exists(target_file) and os.path.exists(f"./data/{target_file}"):
    target_file = f"./data/{target_file}"

if not os.path.exists(target_file):
    print(f"🚨 파일 없음: {target_file}")
    # 디버깅용 파일 목록 출력
    print(f"현재 경로 파일: {os.listdir('.')}")
    raise FileNotFoundError("CSV 파일을 찾을 수 없습니다.")

cols = ['event_time', 'event_type', 'product_id', 'brand', 'price', 'user_id', 'user_session']
df = pd.read_csv(target_file, usecols=cols)
df['event_time'] = pd.to_datetime(df['event_time'], utc=True, errors='coerce')
df = df.dropna(subset=['event_time'])
df = df[df['brand'].isin(['samsung', 'apple', 'xiaomi'])].reset_index(drop=True)

print(f"✅ 데이터 로드 성공: {len(df):,} Rows ({df['event_time'].min().date()} ~ {df['event_time'].max().date()})")


## [Step 3] 피쳐 생성 (Time-Split & Adaptive Logic)

In [ ]:

def create_features_and_target(df_full, cutoff_date_str, lookback_days=30):
    cutoff = pd.Timestamp(cutoff_date_str, tz='UTC')
    start_date = cutoff - pd.Timedelta(days=lookback_days)
    end_date = cutoff + pd.Timedelta(days=7)
    
    print(f"🛠️ 생성: {cutoff.date()} (History {lookback_days}일: {start_date.date()}~)")
    
    mask_past = (df_full['event_time'] >= start_date) & (df_full['event_time'] < cutoff)
    df_past = df_full[mask_past].copy()
    
    mask_future = (df_full['event_time'] >= cutoff) & (df_full['event_time'] < end_date)
    df_future = df_full[mask_future].copy()

    # [Safety] Data Empty
    if df_past.empty:
        print("⚠️ Past Data Empty -> Return Safe Empty DataFrame")
        empty_cols = ['cnt_view', 'cnt_cart', 'cnt_remove', 'cnt_competitor', 'cart_remove_ratio', 
                      'avg_gap_days', 'avg_duration', 'price_tier', 'is_retained']
        return pd.DataFrame(columns=empty_cols)

    # Feature Engineering
    is_samsung = df_past['brand'] == 'samsung'
    df_past['is_view'] = (is_samsung & (df_past['event_type'] == 'view')).astype(int)
    df_past['is_cart'] = (is_samsung & (df_past['event_type'] == 'cart')).astype(int)
    df_past['is_remove'] = (is_samsung & (df_past['event_type'] == 'remove_from_cart')).astype(int)
    df_past['is_competitor'] = (~is_samsung).astype(int)
    
    features = df_past.groupby('user_id').agg(
        cnt_view=('is_view', 'sum'),
        cnt_cart=('is_cart', 'sum'),
        cnt_remove=('is_remove', 'sum'),
        cnt_competitor=('is_competitor', 'sum')
    )
    features['cart_remove_ratio'] = features['cnt_remove'] / (features['cnt_cart'] + 1)
    
    # Gap
    samsung_logs = df_past[is_samsung].sort_values(['user_id', 'event_time'])
    if not samsung_logs.empty:
        s = samsung_logs.groupby(['user_id', 'user_session'])['event_time'].min().reset_index().sort_values(['user_id','event_time'])
        s['prev'] = s.groupby('user_id')['event_time'].shift(1)
        s['gap'] = (s['event_time'] - s['prev']).dt.days
        avg_gap = s.groupby('user_id')['gap'].mean().fillna(lookback_days)
        features = features.join(avg_gap.rename('avg_gap_days'), how='left').fillna(lookback_days)
    else:
        features['avg_gap_days'] = lookback_days
        
    # Duration
    dur = df_past.groupby(['user_id', 'user_session'])['event_time'].agg(lambda x:(x.max()-x.min()).total_seconds())
    features = features.join(dur.groupby('user_id').mean().rename('avg_duration'), how='left').fillna(0)
    
    # Price
    mp = df_past[is_samsung].groupby('user_id')['price'].max().fillna(0)
    features['price_tier'] = pd.cut(mp, bins=[-1, 100, 300, 600, float('inf')], labels=[0,1,2,3]).astype(int)
    
    # Label
    retained = df_future['user_id'].unique()
    features['is_retained'] = 0
    features.loc[features.index.isin(retained), 'is_retained'] = 1
    
    return features.fillna(0)

# Execution (Adaptive)
total_days = (df['event_time'].max() - df['event_time'].min()).days
print(f"📊 총 기간: {total_days}일")

if total_days < 14:
    print("⚠️ Short Data Mode Activated")
    mid = df['event_time'].min() + pd.Timedelta(days=int(total_days/2))
    train_cutoff = mid.strftime('%Y-%m-%d')
    test_cutoff = (df['event_time'].max() - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
    lookback = int(total_days/2)
else:
    test_cutoff = (df['event_time'].max() - pd.Timedelta(days=7)).strftime('%Y-%m-%d')
    train_cutoff = (pd.Timestamp(test_cutoff) - pd.Timedelta(days=7)).strftime('%Y-%m-%d')
    lookback = 30

train_df = create_features_and_target(df, train_cutoff, lookback)
test_df = create_features_and_target(df, test_cutoff, lookback)

# Safety Dummy Row
if len(train_df) == 0:
    print("🚨 Train Data 0건 -> Dummy 추가")
    dummy = pd.DataFrame([[0]*8 + [0]], columns=train_df.columns)
    train_df = pd.concat([train_df, dummy])
    
print(f"✅ 최종 데이터: Train({len(train_df)}), Test({len(test_df)})")


In [ ]:

# Scaling & Clustering
cols_cluster = ['cnt_view', 'cnt_cart', 'avg_gap_days', 'avg_duration', 'cart_remove_ratio', 'price_tier', 'cnt_competitor']
scaler = StandardScaler()

# Dummy Data일 경우 fit 에러 방지 -> 값이 하나면 scale 의미 X
if len(train_df) < 2:
    X_train_s = np.zeros((len(train_df), len(cols_cluster)))
    X_test_s = np.zeros((len(test_df), len(cols_cluster)))
else:
    X_train_s = scaler.fit_transform(train_df[cols_cluster])
    if len(test_df) > 0:
        X_test_s = scaler.transform(test_df[cols_cluster])
    else:
        X_test_s = np.empty((0, len(cols_cluster)))

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
if len(train_df) >= 3:
    train_df['c'] = kmeans.fit_predict(X_train_s)
    # Retention 순서 정렬
    rank = train_df.groupby('c')['is_retained'].mean().sort_values().index
    imap = {old:new for new,old in enumerate(rank)}
    train_df['c'] = train_df['c'].map(imap)
    if len(test_df) > 0:
        test_df['c'] = kmeans.predict(X_test_s)
        test_df['c'] = test_df['c'].map(imap)
else:
    train_df['c'] = 0
    if len(test_df) > 0: test_df['c'] = 0

print("✅ 클러스터링 완료")


In [ ]:

# Tensor Prep
def to_tensor(d, s_scaled):
    if len(d) == 0: return torch.FloatTensor([]), torch.FloatTensor([])
    c_dummy = pd.get_dummies(d['c'], prefix='c').reindex(columns=['c_0','c_1','c_2'], fill_value=0).values
    x = np.hstack([s_scaled, c_dummy])
    y = d['is_retained'].values
    return torch.FloatTensor(x), torch.FloatTensor(y).view(-1, 1)

Xt, yt = to_tensor(train_df, X_train_s)
Xv, yv = to_tensor(test_df, X_test_s)

# Loader
bs = 512
train_loader = DataLoader(TensorDataset(Xt, yt), batch_size=bs, shuffle=True)
if len(Xv) > 0:
    test_loader = DataLoader(TensorDataset(Xv, yv), batch_size=bs, shuffle=False)
else:
    test_loader = []

# Model
class SimpleDNN(nn.Module):
    def __init__(self, d_in):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_in, 64), nn.BatchNorm1d(64), nn.ReLU(),
            nn.Linear(64, 32), nn.BatchNorm1d(32), nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x): return self.net(x)

model = SimpleDNN(Xt.shape[1])
opt = optim.Adam(model.parameters(), lr=0.01)
sche = StepLR(opt, step_size=5, gamma=0.5)
crit = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([10.0]))

loss_hist, f1_hist = [], []

print("▶ 학습 시작")
for ep in range(15):
    model.train()
    e_loss = 0
    for xb, yb in train_loader:
        opt.zero_grad()
        out = model(xb)
        loss = crit(out, yb)
        loss.backward()
        opt.step()
        e_loss += loss.item()
    sche.step()
    
    # Valid
    if len(Xv) > 0:
        model.eval()
        with torch.no_grad():
            out_v = model(Xv)
            prob = torch.sigmoid(out_v).numpy()
            pred = (prob>0.5).astype(int)
            f1 = f1_score(yv.numpy(), pred, zero_division=0)
    else:
        f1 = 0
    
    loss_hist.append(e_loss/len(train_loader))
    f1_hist.append(f1)
    
    if (ep+1)%5==0:
        print(f"Ep {ep+1} | Loss: {loss_hist[-1]:.4f} | F1: {f1:.4f}")

print("✅ 학습 종료")


In [ ]:

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(loss_hist, label='Loss')
plt.title('Loss Curve')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(f1_hist, color='g', label='F1')
plt.title('Validation F1')
plt.legend()
plt.show()

if len(Xv) > 0:
    print(classification_report(yv.numpy(), pred, target_names=['이탈', '유지'], zero_division=0))
